# Creation of CaloChallenge 2022 datasets

In [1]:
import pandas as pd
import numpy as np
import h5py
import glob

## Dataset 1

### Photons

In [2]:
""" Based on high-stats data of https://opendata.cern.ch/record/15012# 
    steps:
        - load csv, take first N events (size of actual GAN training sets)
        - check for nans
        - create dataset in hdf5 file of that energy

"""
folder = '../../../../ML_source/CaloChallenge/photons_samples_highStat/'
num_events = {256: 10000, 512: 10000, 1024: 10000, 2048: 10000, 4096: 10000, 8192: 10000,
              16384: 10000, 32768: 10000, 65536: 10000, 131072: 10000, 262144: 10000, 
              524288: 5000, 1048576: 3000, 2097152: 2000, 4194304: 1000}

dataset_file = h5py.File(folder+'dataset_1_photons.hdf5', 'w')

for n in range(8,23):
    energy = float(2**n)
    file_name = folder+'pid22_E'+str(2**n)+'_eta_20_25_voxalisation.csv'
    loaded_array = pd.read_csv(file_name, header=None).to_numpy()
    if np.isnan(loaded_array[:num_events[energy]]).any():
        raise ValueError("Dataset contains NaNs!")
    dataset_file.create_dataset('data_'+str(int(energy)), data=loaded_array[:num_events[energy]].clip(min=0.),
                               compression='gzip')

dataset_file.close()


### Pions

In [3]:
""" Based on high-stats data of https://opendata.cern.ch/record/15012# 
    steps:
        - load csv, take all given events until high-stats are ready (only events at 4 TeV are missing)
        - check for nans
        - create dataset in hdf5 file of that energy

"""
folder = '../../../../ML_source/CaloChallenge/pion_samples/'

# not used for now:
num_events = {256: 10000, 512: 10000, 1024: 10000, 2048: 10000, 4096: 10000, 8192: 10000,
             16384: 10000, 32768: 10000, 65536: 10000, 131072: 10000, 262144: 10000, 
             524288: 5000, 1048576: 3000, 2097152: 2000, 4194304: 1000}

dataset_file = h5py.File(folder+'dataset_1_pions.hdf5', 'w')

for n in range(8,23):
    energy = float(2**n)
    file_name = folder+'pid211_E'+str(2**n)+'_eta_20_25_voxalisation.csv'
    loaded_array = pd.read_csv(file_name, header=None).to_numpy()
    if np.isnan(loaded_array).any():
        raise ValueError("Dataset contains NaNs!")
    dataset_file.create_dataset('data_'+str(int(energy)), data=loaded_array.clip(min=0.), compression='gzip')

dataset_file.close()

## Dataset 2

### Electrons

In [2]:
""" Based on Dataset2_GPS of Dalila, taken from https://cernbox.cern.ch/index.php/s/KwFvdbub9QNP6qA
    steps:
        - load existing hdf5 files
        - read out energy and shower
        - concatenate to list of 150k showers
        - transform shape (num_events, r_bins, alpha_bins, layer_id) to 
          (num_events, layer_id, alpha_bins, r_bins) as in dataset 1, then flatten last dimensions
        - rescale by sampling_fraction, as given by Anna
        - write to new hdf5 files

"""
folder = '../../../../ML_source/CaloChallenge/Dataset2_cont_energy/'
sampling_fraction = 1./0.033

energy = []
shower = []

output_nr = 1

for idx, source_file in enumerate(glob.glob(folder+'*')):
    data_source = h5py.File(source_file, 'r')

    for key in data_source["Angle_90"].keys():
        energy.append(float(key))
        shower.append(data_source["Angle_90"][key][:])
    data_source.close()
    print("Done with reading file {}/{}".format(idx+1, 120))
    if idx % 60 == 59:
        energy = np.array(energy)
        print(len(energy))
        shower = np.array(shower)
        print(shower.shape)
        shower = np.moveaxis(shower, 3, 1)
        print(shower.shape)
        shower = np.moveaxis(shower, 3, 2)
        print(shower.shape)

        dataset_file = h5py.File(folder + 'dataset_2_{}.hdf5'.format(output_nr), 'w')
        dataset_file.create_dataset('incident_energies', data=energy.clip(min=0.).reshape(len(energy), -1), compression='gzip')
        dataset_file.create_dataset('showers', data=sampling_fraction*shower.clip(min=0.).reshape(len(shower), -1), compression='gzip')
        print("Done with writing file {}".format(output_nr))
        dataset_file.close()
        output_nr += 1
        energy = []
        shower = []


Done with reading file 1/120
Done with reading file 2/120
Done with reading file 3/120
Done with reading file 4/120
Done with reading file 5/120
Done with reading file 6/120
Done with reading file 7/120
Done with reading file 8/120
Done with reading file 9/120
Done with reading file 10/120
Done with reading file 11/120
Done with reading file 12/120
Done with reading file 13/120
Done with reading file 14/120
Done with reading file 15/120
Done with reading file 16/120
Done with reading file 17/120
Done with reading file 18/120
Done with reading file 19/120
Done with reading file 20/120
Done with reading file 21/120
Done with reading file 22/120
Done with reading file 23/120
Done with reading file 24/120
Done with reading file 25/120
Done with reading file 26/120
Done with reading file 27/120
Done with reading file 28/120
Done with reading file 29/120
Done with reading file 30/120
Done with reading file 31/120
Done with reading file 32/120
Done with reading file 33/120
Done with reading f

## Dataset 3

### Electrons

In [2]:
""" Based on Dataset3_GPS of Dalila, taken from https://cernbox.cern.ch/index.php/s/KwFvdbub9QNP6qA
    steps:
        - load existing hdf5 files
        - read out energy and shower
        - concatenate to list of 50k showers
        - transform shape (num_events, r_bins, alpha_bins, layer_id) to 
          (num_events, layer_id, alpha_bins, r_bins) as in dataset 1, then flatten last dimensions
        - rescale by sampling_fraction, as given by Anna
        - write to new hdf5 files

"""

folder = '../../../../ML_source/CaloChallenge/Dataset3_cont_energy/'
sampling_fraction = 1./0.033

energy = []
shower = []

output_nr = 1

for idx, source_file in enumerate(glob.glob(folder+'*')):
    data_source = h5py.File(source_file, 'r')

    for key in data_source["Angle_90"].keys():
        energy.append(float(key))
        shower.append(data_source["Angle_90"][key][:])
    data_source.close()
    print("Done with reading file {}/{}".format(idx+1, 120))
    if idx % 20 == 19:
        energy = np.array(energy)
        print(len(energy))
        shower = np.array(shower)
        print(shower.shape)
        shower = np.moveaxis(shower, 3, 1)
        print(shower.shape)
        shower = np.moveaxis(shower, 3, 2)
        print(shower.shape)

        dataset_file = h5py.File(folder + 'dataset_3_{}.hdf5'.format(output_nr), 'w')
        dataset_file.create_dataset('incident_energies', data=energy.clip(min=0.).reshape(len(energy), -1), compression='gzip')
        dataset_file.create_dataset('showers', data=sampling_fraction*shower.clip(min=0.).reshape(len(shower), -1), compression='gzip')
        print("Done with writing file {}".format(output_nr))
        dataset_file.close()
        output_nr += 1
        energy = []
        shower = []


Done with reading file 1/120
Done with reading file 2/120
Done with reading file 3/120
Done with reading file 4/120
Done with reading file 5/120
Done with reading file 6/120
Done with reading file 7/120
Done with reading file 8/120
Done with reading file 9/120
Done with reading file 10/120
Done with reading file 11/120
Done with reading file 12/120
Done with reading file 13/120
Done with reading file 14/120
Done with reading file 15/120
Done with reading file 16/120
Done with reading file 17/120
Done with reading file 18/120
Done with reading file 19/120
Done with reading file 20/120
50000
(50000, 18, 50, 45)
(50000, 45, 18, 50)
(50000, 45, 50, 18)
Done with writing file 1
Done with reading file 21/120
Done with reading file 22/120
Done with reading file 23/120
Done with reading file 24/120
Done with reading file 25/120
Done with reading file 26/120
Done with reading file 27/120
Done with reading file 28/120
Done with reading file 29/120
Done with reading file 30/120
Done with reading 